# Description

The main goal of this notebook is to help in this cases:
- You need to update the OpenApi file of a Custom Extension.
- You need to create a complete copy of one of your assistants into another assistant. 

The notebook will also create backups of the original assistant in txt and the final txt that will be uploaded. 


!pip install --upgrade "ibm-watson>=7.0.1"

# Description

The goal of these notebook is to sucessfully download the actions and upload them witht he new Custom
extension ids in case we modified them for some reason. 



In [1]:
import pprint
from dotenv import load_dotenv
import os
import time
load_dotenv()

True

# Assistant setup 

In [2]:
URL = 'https://api.us-east.assistant.watson.cloud.ibm.com/'

# this is the IBM cloud IAM apy key. 
API_KEY = os.getenv(key='API_KEY')

# put here the name of the assistant to replace extensions ids 
#ASSISTANT_NAME = 'Experiments_Ask_Proc_Main'

CHANGE_LOG_DESCRIPTION = '''
----- Change Description -----

Assistant: DEV_TEST_AskProc
Change: Epm Api extension from V21 to V22
No change needed to watsonx api 

----- Change Description -----

DEV_TEST_AskProc
epm_api_v21 - catalogid: "9b375f7a-96f4-4d11-a64b-38c5d3886674"
epm_api_v21 - hashid: "0710d532e446af11a39ab1c8d3d4b5281b94cfd91550eb8f2cf6d9ff669b3290"

epm_api_v22 - catalogid: "9c8978e5-39b8-466b-a22a-ed070fc57648"
epm_api_v22 - hashid: "b9a5e2cf711d38ba9985557b7da7f7aa934b1c3a372461453e26f31e062ff682"

'''

# available assistants 
'''
DEV_TEST_AskProc
PREPROD_PROD_AskProc
'''

# in case you are just updating the OpenApi file you have to use the same assistant name here in the two variables. 
ASSISTANT_FROM = 'DEV_TEST_AskProc'
ASSISTANT_TO = 'DEV_TEST_AskProc'

BACKUP_FOLDER = 'assistant_backups/'

ONLY_EPM_API_CHANGE = True


'''
DEV_TEST_AskProc
epm_api_v21 - catalogid: "9b375f7a-96f4-4d11-a64b-38c5d3886674"
epm_api_v21 - hashid: "0710d532e446af11a39ab1c8d3d4b5281b94cfd91550eb8f2cf6d9ff669b3290"

epm_api_v22 - catalogid: "9c8978e5-39b8-466b-a22a-ed070fc57648"
epm_api_v22 - hashid: "b9a5e2cf711d38ba9985557b7da7f7aa934b1c3a372461453e26f31e062ff682"

watsonx_api - catalogid:
watsonx_api - hashid: 

PREPROD_PROD_AskProc

'''

# You can get this ids by instecting the browser and going to the Network tab. This is done when you click on one of the Integrations 
# so you can see the Catalog id and hash id of the extension. 
MODIFY = [
    {
        'id_from': "9b375f7a-96f4-4d11-a64b-38c5d3886674", #  epm ask_proc_main
        'id_to': "9c8978e5-39b8-466b-a22a-ed070fc57648" # epm_api_v21 ask_proc_main
        },
        {
        'id_from': "0710d532e446af11a39ab1c8d3d4b5281b94cfd91550eb8f2cf6d9ff669b3290", #   epm ask_proc_main
        'id_to': "b9a5e2cf711d38ba9985557b7da7f7aa934b1c3a372461453e26f31e062ff682" # epm_api_v21 ask_proc_main
        }
]

In [3]:
# Auhentification 
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV2(
    version='2021-06-14',
    authenticator = authenticator
)

assistant.set_service_url(URL)



# Create folder for backup for data and logs 

In [ ]:
# get current datetime in string
import datetime as dt
date_name = dt.datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
folder_name = date_name
BACKUP_FOLDER = BACKUP_FOLDER+folder_name
date_name

In [ ]:
os.makedirs(BACKUP_FOLDER)

In [ ]:
BACKUP_FOLDER

In [ ]:
logs_path = BACKUP_FOLDER+'/CHANGE_LOG_DESCRIPTION.txt'
with open(logs_path, mode='w') as f:
    f.write(CHANGE_LOG_DESCRIPTION)
    f.close()

# Changes setup 
In our case we only have 2 custom expensions and only one is the most commun to have changes so that is why I have to flows 
One when I only want to change the ids for EPM api And other when I want to change epm and watsonx.ai ids 


# Getting the assistant ID using the assistant name

In [4]:
assistant_list = assistant.list_assistants()

In [5]:
# find your assistant by name 


for bot in assistant_list.get_result().get('assistants'):
    if bot.get('name') == ASSISTANT_FROM:
        ASSISTANT_FROM_ID = bot.get('assistant_id')
        pprint.pprint(bot)
    if bot.get('name') == ASSISTANT_TO:
        ASSISTANT_TO_ID = bot.get('assistant_id')
        pprint.pprint(bot)

print(f'\n\nYour assistant FROM id is: {ASSISTANT_FROM_ID}')
print(f'\n\nYour assistant TO id is: {ASSISTANT_TO_ID}')

{'assistant_environments': [{'environment': 'live',
                             'environment_id': '3d6221bd-3902-437b-b8fe-61659530d549',
                             'name': 'live'},
                            {'environment': 'draft',
                             'environment_id': '8b2d321a-d698-42d5-a8a6-afadb7630aac',
                             'name': 'draft'}],
 'assistant_id': 'bf5deaec-5f82-47a7-8d75-6b843b3190af',
 'assistant_skills': [{'skill_id': '9a0cd298-cf7c-4bc1-b7a2-3669759ee53e',
                       'type': 'dialog'},
                      {'skill_id': '8ed9e13a-7bc1-4ae0-babd-06080309ad90',
                       'type': 'action'}],
 'description': '',
 'language': 'en',
 'multi_lingual': {'enabled': False},
 'name': 'DEV_TEST_AskProc'}
{'assistant_environments': [{'environment': 'live',
                             'environment_id': '3d6221bd-3902-437b-b8fe-61659530d549',
                             'name': 'live'},
                            {'environment': 

# Get the skills data from your assistant.

In [6]:
'''
A successful call to this method only initiates an asynchronous export. 
The exported JSON data is not available until processing completes.

After the initial request is submitted, you can poll the status of the operation by calling the same request again and 
checking the value of the status property. If an error occurs (indicated by a status value of Failed), 
the status_description property provides more information about the error, and the status_errors property 
contains an array of error messages that caused the failure.

When processing has completed, the request returns the exported JSON data. Remember that the usual rate limits apply.
'''
while True:
    time.sleep(2)
    skills_from = assistant.export_skills(assistant_id=ASSISTANT_FROM_ID)
    assistant_from_data = skills_from.result
    if assistant_from_data.get('status') != 'Processing':
        break

In [8]:
# here you get the json data 
assistant_from_data.keys()

dict_keys(['assistant_skills', 'assistant_state'])

In [ ]:
assistant_from_data.keys()

# Transform the assistant_skills data into text 

In [ ]:
assistant_from_skills_txt = str(assistant_from_data['assistant_skills'])

In [ ]:

file_name = BACKUP_FOLDER+'/'+ASSISTANT_FROM+'_DOWNLOAD_'+date_name+'.txt'
print(file_name)

with open(file=file_name, mode='w') as f:
    f.write(assistant_from_skills_txt)
    f.close()

# Process to replace strings in assistant 

With the data in string format you can now use the replace method to replace the catalog and hash ids 

In [ ]:
for i in MODIFY:
    assistant_from_skills_txt = assistant_from_skills_txt.replace(i['id_from'], i['id_to'])

# After replacement 

Once the ids are replaced , now we can transform the text again to a list using eval function. 

In [ ]:
# saving the file as upload 
upload_name = BACKUP_FOLDER+'/'+ASSISTANT_FROM+'_UPLOAD_'+date_name+'.txt'
print(upload_name)

with open(file=upload_name, mode='w') as f:
    f.write(assistant_from_skills_txt)
    f.close()

In [ ]:
# then we can go back to list here.
assistant_from_skills_list = eval(assistant_from_skills_txt)

# Upload updated skills 

Now we can upload the skills data again in the server 

In [ ]:
import_skills = assistant.import_skills(assistant_id=ASSISTANT_TO_ID, 
                                        include_audit=True, 
                                        assistant_skills=assistant_from_skills_list, 
                                        assistant_state = assistand_from_data['assistant_state'])

# Check the import state 

In [ ]:
assistant.import_skills_status(assistant_id=ASSISTANT_TO_ID).get_result()